# Example 1: Multiple-response optimization of the acidic treatment of the brown alga Ecklonia radiata for the sequential extraction of fucoidan and alginate

In the [original article](https://www.sciencedirect.com/science/article/pii/S0960852415012092), the authors optimized the acidic treatment of the brown alga Ecklonia radiata in order to extract [fucoidan](https://en.wikipedia.org/wiki/Fucoidan) and facilitate the efficient sequential extraction of alginates.

They investigated the effects of

* temperature
* pH
* and duration of the acidic treatment

on fucoidan yield, alginate extractability, and the molecular weight of sequentially extracted alginates.

![](https://ars.els-cdn.com/content/image/1-s2.0-S0960852415012092-fx1.jpg)

Here, we use the data from the article to reproduce their models.

In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from mpl_toolkits.mplot3d import axes3d
from matplotlib import pyplot as plt
plt.style.use('seaborn-notebook')

In [ ]:
tabell = """Temperature (°C)	x1	pH	x2	Duration (min)	x3	FRPF yieldc (%)	Alginate yieldd (%)	Alginate MWe (kDa)
35	0	3.5	0	60	0	3.19	33.7	709
35	0	3.5	0	60	0	3.05	33.2	719
35	0	3.5	0	60	0	2.98	34.7	746
25	-1	3.5	0	10	-1	2.31	32.2	736
25	-1	3.5	0	110	1	3.10	33.3	734
45	1	3.5	0	10	-1	2.69	33.1	728
45	1	3.5	0	110	1	3.00	34.5	798
35	0	2	-1	10	-1	2.65	35.0	672
35	0	2	-1	110	1	3.03	38.0	663
35	0	5	1	10	-1	2.92	35.0	677
35	0	5	1	110	1	3.09	33.7	693
25	-1	2	-1	60	0	2.86	36.4	514
25	-1	5	1	60	0	2.93	33.1	616
45	1	2	-1	60	0	3.03	38.3	690
45	1	5	1	60	0	2.95	35.1	586"""

In [ ]:
header = []
numbers = []
for i, row in enumerate(tabell.split('\n')):
    split = row.split('\t')
    if i == 0:
        header = [key for key in split]
    else:
        numbers.append([float(j) for j in split])
numbers = np.array(numbers)
tabell_dict = {}
for i, key in enumerate(header):
    tabell_dict[key] = numbers[:, i]
data = pd.DataFrame.from_dict(tabell_dict)

In [ ]:
data

The models created were on the form:

\begin{equation}
Y = a + b_1 x_1 + b_2 x_2 + b_3 x_3 + b_{12} x_1 x_2 + b_{13} x_1 x_3 + b_{23} x_2 x_3 + b_{11} x_1^2 + b_{22} x_2^2 + b_{33} x_3^2
\end{equation}

In [ ]:
data[['x1', 'x2', 'x3']]

In [ ]:
x1_x2_x3 = data[['x1', 'x2', 'x3']].to_numpy()
y1 = data['FRPF yieldc (%)'].to_numpy()
y2 = data['Alginate yieldd (%)'].to_numpy()
y3 = data['Alginate MWe (kDa)'].to_numpy()

In [ ]:
# Make X matrix, consisting of [1, x1, x2, x3, x1^2, x1x2, ...] etc. for fitting:
poly = PolynomialFeatures(degree=2)  # second order polynomial & interactions
X = poly.fit_transform(x1_x2_x3)
# print out the columns of X:
print('Columns in X are:', poly.get_feature_names_out())

In [ ]:
model1 = LinearRegression(fit_intercept=False)
model1.fit(X, y1)
print(model1.coef_)

In [ ]:
model2 = LinearRegression(fit_intercept=False)
model2.fit(X, y2)
print(model2.coef_)

In [ ]:
model3 = LinearRegression(fit_intercept=False)
model3.fit(X, y3)
print(model3.coef_)

In [ ]:
coefficients = {
    'a': 0,
    'b1': 1,
    'b2': 2,
    'b3': 3,
    'b12': 5,
    'b13': 6,
    'b23': 8,
    'b11': 4,
    'b22': 7,
    'b33': 9,
}
# Make table similar to table 3:
table3 = {
    'y': ['FRPF yieldc (%)', 'Alginate yieldd (%)', 'Alginate MWe (kDa)'],
}
for i, idx in coefficients.items():
    table3[i] = [model1.coef_[idx], model2.coef_[idx], model3.coef_[idx]]
table3 = pd.DataFrame.from_dict(table3)
table3

In [ ]:
# Reproduce fig 2, at T = 35 C, x1 = 0
pH = np.linspace(2, 5, 25)
dur = np.linspace(10, 110, len(pH))
# Transform pH -> x2, dur -> x3
x2 = ((1-(-1))/(5-2)) * (pH - 2) + (-1)
x3 = ((1-(-1))/(110-10)) * (dur - 10) + (-1)
x1 = 0
# Create matrices for evaluating surfaces:
X2, X3 = np.meshgrid(x2, x3)
X1 = np.zeros_like(X2)

In [ ]:
# Evaluate the models using the fitted objects:
x1_x2_x3_p = np.c_[X1.ravel(), X2.ravel(), X3.ravel()]
X = poly.fit_transform(x1_x2_x3_p)

Y1 = model1.predict(X).reshape(X1.shape)
Y2 = model2.predict(X).reshape(X1.shape)
Y3 = model3.predict(X).reshape(X1.shape)

In [ ]:
# Plot Y1, Y2, and Y3:
fig1, ax1 = plt.subplots(constrained_layout=True, subplot_kw={'projection': '3d'})
pH_mat, dur_mat = np.meshgrid(pH, dur)
# Plot the 3D surface
cset = ax1.contour(pH_mat, dur_mat, Y1, zdir='z', offset=2.4, levels=10)
ax1.plot_surface(pH_mat, dur_mat, Y1, cmap='viridis')
ax1.set_xlabel('pH')
ax1.set_ylabel('Duration')
ax1.set_zlabel('FRPF yieldc (%)')
ax1.set_zlim(2.4, 3.2)
ax1.set_xlim(2, 5)
ax1.set_ylim(10, 110)
ax1.xaxis.pane.fill = False
ax1.yaxis.pane.fill = False
ax1.zaxis.pane.fill = False

In [ ]:
fig1, ax1 = plt.subplots(constrained_layout=True, subplot_kw={'projection': '3d'})
pH_mat, dur_mat = np.meshgrid(pH, dur)
# Plot the 3D surface
cset = ax1.contour(pH_mat, dur_mat, Y2, zdir='z', offset=32, levels=10)
ax1.plot_surface(pH_mat, dur_mat, Y2, cmap='viridis')
ax1.set_xlabel('pH')
ax1.set_ylabel('Duration')
ax1.set_zlabel('Alginate yieldd (%)')
ax1.set_zlim(32, 39)
ax1.set_xlim(2, 5)
ax1.set_ylim(10, 110)
ax1.xaxis.pane.fill = False
ax1.yaxis.pane.fill = False
ax1.zaxis.pane.fill = False

In [ ]:
fig1, ax1 = plt.subplots(constrained_layout=True, subplot_kw={'projection': '3d'})
pH_mat, dur_mat = np.meshgrid(pH, dur)
# Plot the 3D surface
cset = ax1.contour(pH_mat, dur_mat, Y3, zdir='z', offset=500, levels=10)
ax1.plot_surface(pH_mat, dur_mat, Y3, cmap='viridis')
ax1.set_xlabel('pH')
ax1.set_ylabel('Duration')
ax1.set_zlabel('Alginate MWe (kDa)')
ax1.set_zlim(500, 800)
ax1.set_xlim(2, 5)
ax1.set_ylim(10, 110)
ax1.xaxis.pane.fill = False
ax1.yaxis.pane.fill = False
ax1.zaxis.pane.fill = False

In [ ]:
# Evaluate models "by hand":

coeff = {key: model1.coef_[idx] for key, idx in coefficients.items()}
Y1 = (
    coeff['a'] +
    coeff['b1'] * x1 +
    coeff['b2'] * X2 +
    coeff['b3'] * X3 +
    coeff['b12'] * x1 * X2 +
    coeff['b13'] * x1 * X3 +
    coeff['b23'] * X2 * X3 +
    coeff['b11'] * x1**2 +
    coeff['b22'] * X2**2 +
    coeff['b33'] * X3**2
)
coeff = {key: model2.coef_[idx] for key, idx in coefficients.items()}
Y2 = (
    coeff['a'] +
    coeff['b1'] * x1 +
    coeff['b2'] * X2 +
    coeff['b3'] * X3 +
    coeff['b12'] * x1 * X2 +
    coeff['b13'] * x1 * X3 +
    coeff['b23'] * X2 * X3 +
    coeff['b11'] * x1**2 +
    coeff['b22'] * X2**2 +
    coeff['b33'] * X3**2
)
coeff = {key: model3.coef_[idx] for key, idx in coefficients.items()}
Y3 = (
    coeff['a'] +
    coeff['b1'] * x1 +
    coeff['b2'] * X2 +
    coeff['b3'] * X3 +
    coeff['b12'] * x1 * X2 +
    coeff['b13'] * x1 * X3 +
    coeff['b23'] * X2 * X3 +
    coeff['b11'] * x1**2 +
    coeff['b22'] * X2**2 +
    coeff['b33'] * X3**2
)

In [ ]:
fig1, (ax1, ax2, ax3) = plt.subplots(
    constrained_layout=True,
    subplot_kw={'projection': '3d'},
    ncols=3,
    figsize=(9, 3),
)
pH_mat, dur_mat = np.meshgrid(pH, dur)

# Plot the 3D surface for Y1:
cset = ax1.contour(pH_mat, dur_mat, Y1, zdir='z', offset=2.4, levels=10)
ax1.plot_surface(pH_mat, dur_mat, Y1, cmap='viridis')
ax1.set_xlabel('pH')
ax1.set_ylabel('Duration')
ax1.set_zlabel('FRPF yieldc (%)')
ax1.set_zlim(2.4, 3.2)
ax1.set_xlim(2, 5)
ax1.set_ylim(10, 110)
ax1.xaxis.pane.fill = False
ax1.yaxis.pane.fill = False
ax1.zaxis.pane.fill = False


# Plot the 3D surface for Y2:
cset = ax2.contour(pH_mat, dur_mat, Y2, zdir='z', offset=32, levels=10)
ax2.plot_surface(pH_mat, dur_mat, Y2, cmap='viridis')
ax2.set_xlabel('pH')
ax2.set_ylabel('Duration')
ax2.set_zlabel('Alginate yieldd (%)')
ax2.set_zlim(32, 39)
ax2.set_xlim(2, 5)
ax2.set_ylim(10, 110)
ax2.xaxis.pane.fill = False
ax2.yaxis.pane.fill = False
ax2.zaxis.pane.fill = False


# Plot the 3D surface for Y3
cset = ax3.contour(pH_mat, dur_mat, Y3, zdir='z', offset=500, levels=10)
ax3.plot_surface(pH_mat, dur_mat, Y3, cmap='viridis')
ax3.set_xlabel('pH')
ax3.set_ylabel('Duration')
ax3.set_zlabel('Alginate MWe (kDa)')
ax3.set_zlim(500, 800)
ax3.set_xlim(2, 5)
ax3.set_ylim(10, 110)
ax3.xaxis.pane.fill = False
ax3.yaxis.pane.fill = False
ax3.zaxis.pane.fill = False